In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('./data/mixed_first_marketplace.sqlite3')

cursor = conn.cursor()

# cursor.execute("SELECT feedbacks.order_amount_usd, items.total_sales FROM feedbacks, items WHERE items.hash_str = feedbacks.item_hash")

# for each item with unique item_hash
# we have the total_sales
# for each feedback, we have the transaction amount
# sum all the transaction amount from the feedbacks with the same item_hash
# divide the total sales, this should give per-item-price

cursor.execute("SELECT item_hash, SUM(order_amount_usd) AS total_order_amount \
               FROM feedbacks \
               WHERE (marketplace = 'Agora' OR marketplace = 'Evolution') \
               GROUP BY item_hash")

result = cursor.fetchall()

column_names = [description[0] for description in cursor.description]
dff = pd.DataFrame(result, columns=column_names)
dff.to_csv("./data/total_order_amount_by_item.csv", index=False)

cursor.close()
conn.close()


In [32]:
# code for new output
import pandas as pd
import csv
from tqdm import tqdm

df = pd.read_csv("./data/mixed_first_items_merged.csv")
column_names = df.columns
print(column_names)

columns_to_keep = ['marketplace', 'item_id', 'date', 'sales', 'vendor_name', 'author', 'ships_from_coded', 'ships_to', 'category', 'if_drug', 'top_category', '2nd_level_category']

df = df[columns_to_keep]

df = df[df['marketplace'].isin(['agora', 'evolution'])]

df1 = pd.read_csv("./data/total_order_amount_by_item.csv")

merged_df = df.merge(df1, left_on='item_id', right_on='item_hash', how='left')

# Calculate the 'price' column
merged_df['price'] = merged_df['total_order_amount'] / merged_df['sales']


df2 = pd.read_csv("./data/mixed_first_items_2_market.csv")


merged_df = merged_df.merge(df2[['hash_str', 'title']], left_on='item_id', right_on='hash_str', how='left')

merged_df.to_csv("new_output_test.csv", index=False)

print(merged_df.shape)

df3 = pd.read_csv("./data/Agora_Evolution_matching_map.csv").drop_duplicates()

merged_df = merged_df.merge(pd3[['title', 'drug_name_offline']], left_on='title', right_on = 'title' , how='left')

# merged_df = merged_df.drop_duplicates()

# names = []

# with tqdm(total=df.shape[0]) as pbar:  
#     for index, row in merged_df.iterrows():
#         title = row['title']
        
#         matching_row = df3[df3['title'] == title]
        
#         if not matching_row.empty:
#             name = matching_row.iloc[0]['drug_name_offline']
#             names.append(name)
#         else:
#             # Handle cases where there is no match
#             names.append(None)
#     pbar.update(1)

# merged_df['drug_name_offline'] = names

print(merged_df.shape)


merged_df.to_csv("new_output.csv", index=False)

Index(['review_id', 'marketplace', 'item_id', 'date', 'sales',
       'feedback_value', 'vendor_name', 'author', 'ships_from_coded',
       'ships_to', 'category', 'if_drug', 'top_category',
       '2nd_level_category'],
      dtype='object')
(698518, 17)
(699129, 18)


In [19]:
# save the files seperateky
df = pd.read_csv("new_output.csv")

agora_df = df[df['marketplace'].isin(['agora'])]
evolution_df = df[df['marketplace'].isin(['evolution'])]

agora_df.to_csv("Agora_new_output.csv", index=False)
evolution_df.to_csv("Evolution_new_output.csv", index=False)

In [ ]:
['marketplace', 'item_id', 'date', 'sales', 'price', 'vendor_name', 'author', 
 'ships_from_coded', 'ships_to', 'category', 'if_drug', 'top_category', '2nd_level_category']

# marketplace -> name of the market
# item id -> mixed_first_items_merged.csv : item_id
# date -> mixed_first_items_merged.csv : date
# sales -> mixed_first_items_merged.csv : sales
# price -> sqlite3 : feedbacks : order
# vendor_name -> mixed_first_items_merged.csv : 
# author -> mixed_first_items_merged.csv : 
# ships_from_coded -> mixed_first_items_merged.csv : 
# ships_to -> mixed_first_items_merged.csv : 
# category -> mixed_first_items_merged.csv : 
# if_drug -> mixed_first_items_merged.csv : 
# top_category -> mixed_first_items_merged.csv : 
# 2nd_level_category -> mixed_first_items_merged.csv : 

In [65]:
# code for feedback_matched

df = pd.read_csv("./data/mixed_first_items_merged.csv")

# filter out the 2 marketplaces
df = df[df['marketplace'].isin(['agora', 'evolution'])]

print(df.columns)

# get the title of the drug before getting the offline name
df2 = pd.read_csv("./data/mixed_first_items_2_market.csv")

merged_df = df.merge(df2[['hash_str', 'title']], left_on='item_id', right_on='hash_str', how='left')

print(merged_df.columns)

df3 = pd.read_csv("./data/Agora_Evolution_matching_map.csv").drop_duplicates()

merged_df = merged_df.merge(pd3[['title', 'drug_name_offline']], left_on='title', right_on = 'title' , how='left')

merged_df = merged_df.drop(['hash_str', 'title'], axis=1)

# merged_df.to_csv("test.csv", index=False)
agora_feedback = merged_df[merged_df['marketplace'].isin(['agora'])]
evolution_feedback = merged_df[merged_df['marketplace'].isin(['evolution'])]

agora_feedback.to_csv("Agora_feedback_matched.csv", index=False)
evolution_feedback.to_csv("Evolution_feedback_matched.csv", index=False)


Index(['review_id', 'marketplace', 'item_id', 'date', 'sales',
       'feedback_value', 'vendor_name', 'author', 'ships_from_coded',
       'ships_to', 'category', 'if_drug', 'top_category',
       '2nd_level_category'],
      dtype='object')
Index(['review_id', 'marketplace', 'item_id', 'date', 'sales',
       'feedback_value', 'vendor_name', 'author', 'ships_from_coded',
       'ships_to', 'category', 'if_drug', 'top_category', '2nd_level_category',
       'hash_str', 'title'],
      dtype='object')
